In [8]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys
from tqdm.notebook import tqdm
root = '../'

In [5]:
start_df = pd.read_csv(root+'trainfiles/start.csv')
end_df = pd.read_csv(root+'trainfiles/end.csv')

In [10]:
segments_df = start_df.append(end_df)
speakers = Counter(segments_df['speaker'])
common_speakers = speakers.most_common(20)
common_speakers = [common_speaker[0] for common_speaker in common_speakers]
segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker] for speaker in common_speakers}
segments_by_speakers

,end,segfile,segment_idx,silence_ratio,soundfile,speaker,start
0,680.113,trainfiles/start_segments/p960712_seg0.npy,0.0,0.050,trainfiles/LDC97S66/96_dev/devdata/p960712.sph,mary_ambrose,640.611
1,700.922,trainfiles/start_segments/p960712_seg1.npy,1.0,0.140,trainfiles/LDC97S66/96_dev/devdata/p960712.sph,nenet_shevek,680.337
2,717.067,trainfiles/start_segments/p960712_seg2.npy,2.0,0.220,trainfiles/LDC97S66/96_dev/devdata/p960712.sph,p960712_johndoe001,701.393
3,728.312,trainfiles/start_segments/p960712_seg3.npy,3.0,0.065,trainfiles/LDC97S66/96_dev/devdata/p960712.sph,nenet_shevek,717.213
4,747.117,trainfiles/start_segments/p960712_seg4.npy,4.0,0.050,trainfiles/LDC97S66/96_dev/devdata/p960712.sph,p960712_johndoe001,728.831
...,...,...,...,...,...,...,...
20696,3419.851,trainfiles/end_segments/ee970701_seg200.npy,200.0,0.050,trainfiles/LDC98S71/H4E97_01/ee970701.sph,ee970701_spkr_33,3409.412
20697,3437.358,trainfiles/end_segments/ee970701_seg202.npy,202.0,0.000,trainfiles/LDC98S71/H4E97_01/ee970701.sph,ee970701_spkr_34,3433.166
20698,3444.256,trainfiles/end_segments/ee970701_seg203.npy,203.0,0.000,trainfiles/LDC98S71/H4E97_01/ee970701.sph,ee970701_spkr_35,3437.358
20699,3448.491,trainfiles/end_segments/ee970701_seg204.npy,204.0,0.000,trainfiles/LDC98S71/H4E97_01/ee970701.sph,rusty_dornin,3444.256


## Maybe try increasing dataset size by doing each combination of 290*290

In [13]:
overlay = []
for first_speaker in tqdm(common_speakers):
    second_speakers = [speaker for speaker in common_speakers if speaker!=first_speaker ]
    first_segments = segments_by_speakers[first_speaker]
    for second_speaker in second_speakers:
        second_segments = segments_by_speakers[second_speaker]
        third_speakers = [speaker for speaker in common_speakers if speaker not in [first_speaker, second_speaker]]
        for third_speaker in third_speakers:
            third_segments = segments_by_speakers[third_speaker]
            first_idx = np.arange(290)
            second_idx = np.arange(290)
            third_idx = np.arange(290)
            np.random.shuffle(first_idx)
            np.random.shuffle(second_idx)
            np.random.shuffle(third_idx)
            for i in range(290):
                first_segment = first_segments.iloc[first_idx[i]]
                second_segment = second_segments.iloc[second_idx[i]]
                third_segment = third_segments.iloc[third_idx[i]]
                row = {'first_speaker': first_speaker, 'second_speaker': second_speaker, 'third_speaker': third_speaker,
                      'first_file': first_segment['segfile'], 'second_file': second_segment['segfile'], 'third_file': third_segment['segfile']}
                overlay.append(row)
overlay = pd.DataFrame(overlay)

In [14]:
overlay_shuffled = overlay.sample(frac=1, random_state = 12345)
tenth = int(len(overlay)*0.1)
overlay_shuffled[:8*tenth].to_csv('overlay-train.csv')
overlay_shuffled[8*tenth:9*tenth].to_csv('overlay-val.csv')
overlay_shuffled[9*tenth:].to_csv('overlay-test.csv')
overlay_shuffled

,first_speaker,second_speaker,third_speaker,first_file,second_file,third_file
1824810,robert_siegel,natalie_allen,linda_wertheimer,trainfiles/end_segments/j960617__seg206.npy,trainfiles/start_segments/e960516__seg96.npy,trainfiles/start_segments/j960520b_seg2.npy
1871141,robert_siegel,csp_waj_susan,brian_lamb,trainfiles/end_segments/j960607a_seg61.npy,trainfiles/end_segments/i960611__seg94.npy,trainfiles/start_segments/i960610__seg170.npy
644009,ted_koppel,eddie_mair,natalie_allen,trainfiles/start_segments/a960611__seg24.npy,trainfiles/start_segments/eh971023_seg39.npy,trainfiles/start_segments/ee970627_seg141.npy
1903992,andrea_arsenault,lou_waters,mark_mullen,trainfiles/end_segments/d960515__seg294.npy,trainfiles/start_segments/ee970703_seg113.npy,trainfiles/end_segments/b960531a_seg53.npy
1881224,robert_siegel,andrea_arsenault,ted_koppel,trainfiles/end_segments/j960613c_seg44.npy,trainfiles/end_segments/d960604a_seg2.npy,trainfiles/start_segments/a960522__seg35.npy
...,...,...,...,...,...,...
1396132,mark_mullen,linden_soles,lisa_mullins,trainfiles/start_segments/b960611b_seg16.npy,trainfiles/start_segments/g960529__seg53.npy,trainfiles/start_segments/eh971021_seg16.npy
1443969,mark_mullen,eddie_mair,kathleen_kennedy,trainfiles/start_segments/b960605b_seg21.npy,trainfiles/start_segments/eh971021_seg117.npy,trainfiles/start_segments/h960520__seg94.npy
1965341,andrea_arsenault,lynn_vaughan,lisa_mullins,trainfiles/start_segments/d960515__seg88.npy,trainfiles/start_segments/f960614__seg24.npy,trainfiles/start_segments/eh971021_seg63.npy
77285,peter_jennings,lynn_vaughan,csp_waj_susan,trainfiles/start_segments/ea980109_seg32.npy,trainfiles/start_segments/f960703__seg35.npy,trainfiles/start_segments/i960611__seg229.npy
